In [ ]:
import os
xla_flags = os.environ.get('XLA_FLAGS', '')
xla_flags += ' --xla_gpu_triton_gemm_any=True'
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ['XLA_FLAGS'] = xla_flags
# os.environ["MUJOCO_GL"] = "egl"
os.environ['JAX_PLATFORM_NAME'] = 'gpu'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # change to your GPU id
import pickle
from datetime import datetime
from helper import parse_cfg
from omegaconf import OmegaConf
from flax.training import checkpoints

# import mediapy as media
import hydra
import functools
from brax.training.acme import running_statistics
from etils import epath
from omegaconf import OmegaConf
from typing import Any, Callable, Dict, Optional, Type, Union, Tuple
from mujoco import mjx
import numpy
from custom_envs import registry, dm_control_suite, locomotion
import jax
from learning.agents.ppo import networks as ppo_networks
from learning.configs import dm_control_training_config, locomotion_training_config


In [2]:
from learning.module.wrapper.wrapper import Wrapper
class BraxDomainRandomizationWrapper(Wrapper):
  """Brax wrapper for domain randomization."""
  def __init__(
      self,
      env: mjx_env.MjxEnv,
      randomization_fn: Callable[[mjx.Model], Tuple[mjx.Model, mjx.Model]],
  ):
    super().__init__(env)
    self._mjx_model, self._in_axes = randomization_fn(self.env.mjx_model)
    self.env.unwrapped._mjx_model = self._mjx_model

  # def _env_fn(self, mjx_model: mjx.Model) -> mjx_env.MjxEnv:
  #   env = self.env
  #   env.unwrapped._mjx_model = mjx_model
  #   return env

  def reset(self, rng: jax.Array) -> mjx_env.State:
    # def reset(mjx_model, rng):
    #   env = self._env_fn(mjx_model=mjx_model)
    #   return env.reset(rng)

    state = self.env.reset(rng)
    return state

  def step(self, state: mjx_env.State, action: jax.Array) -> mjx_env.State:
    # def step(mjx_model, s, a):
    #   env = self._env_fn(mjx_model=mjx_model)
    #   return env.step(s, a)

    res = self.env.step(state, action)
    return res


In [3]:
CAMERAS = {
    "AcrobotSwingup": "fixed",
    "AcrobotSwingupSparse": "fixed",
    "BallInCup": "cam0",
    "CartpoleBalance": "fixed",
    "CartpoleBalanceSparse": "fixed",
    "CartpoleSwingup": "fixed",
    "CartpoleSwingupSparse": "fixed",
    "CheetahRun": "side",
    "FingerSpin": "cam0",
    "FingerTurnEasy": "cam0",
    "FingerTurnHard": "cam0",
    "FishSwim": "fixed_top",
    "HopperHop": "cam0",
    "HopperStand": "cam0",
    "HumanoidStand": "side",
    "HumanoidWalk": "side",
    "HumanoidRun": "side",
    "PendulumSwingup": "fixed",
    "PointMass": "cam0",
    "ReacherEasy": "fixed",
    "ReacherHard": "fixed",
    "SwimmerSwimmer6": "tracking1",
    "WalkerRun": "side",
    "WalkerWalk": "side",
    "WalkerStand": "side",    
    "Go1Handstand": "side",
    "Go1JoystickRoughTerrain": "track",
}


In [ ]:

from agents.flowsac import networks as flowsac_networks
from agents.rambo import networks as rambo_networks

x_data, y_data, y_dataerr = [], [], []
times = [datetime.now()]

def evaluate(cfg):

    # Load environment
    env = registry.load(cfg.task)
    env_cfg = registry.get_default_config(cfg.task)
    print("env nbody", env._mj_model.nbody)
    print("env nv", env._mj_model.nv)
    obs_size = env.observation_size
    act_size = env.action_size
    rng = jax.random.PRNGKey(cfg.eval_seed)
    
    if cfg.randomization:
        path = epath.Path(".").resolve()

        randomization_fn = registry.get_domain_randomizer_eval(cfg.task)
        rng, dynamics_rng = jax.random.split(rng)
        randomization_fn = functools.partial(randomization_fn, rng = dynamics_rng, params=env.dr_range)
        env = BraxDomainRandomizationWrapper(
            env,
            randomization_fn=randomization_fn,
        )
    if cfg.policy == "sac":
        if cfg.task in dm_control_suite._envs:
            sac_params = dm_control_training_config.brax_sac_config(cfg.task)
        elif cfg.task in locomotion._envs:
            sac_params = locomotion_training_config.locomotion_sac_config(cfg.task)
        sac_training_params = dict(sac_params)
        network_factory = sac_networks.make_sac_networks
        if "network_factory" in sac_params:
            del sac_training_params["network_factory"]
            network_factory = functools.partial(
                sac_networks.make_sac_networks,
                **sac_params.network_factory
            )
        sac_network = network_factory(
            observation_size=obs_size,
            action_size=act_size,
            preprocess_observations_fn=running_statistics.normalize if sac_params.normalize_observations else None, 
        )
        make_policy_fn = sac_networks.make_inference_fn(sac_network)
    elif cfg.policy == "ppo":
        if cfg.task in dm_control_suite._envs:
            ppo_params = dm_control_training_config.brax_ppo_config(cfg.task)
        elif cfg.task in locomotion._envs:
            ppo_params = locomotion_training_config.locomotion_ppo_config(cfg.task)
        network_factory = ppo_networks.make_ppo_networks
        if "network_factory" in ppo_params:
            network_factory = functools.partial(
                ppo_networks.make_ppo_networks,
                **ppo_params.network_factory
            )
        ppo_network = network_factory(
            observation_size=obs_size,
            action_size=act_size,
            preprocess_observations_fn=running_statistics.normalize if ppo_params.normalize_observations else None,
        )

        make_policy_fn = ppo_networks.make_inference_fn(ppo_network)
    elif cfg.policy == "rambo":
        rambo_params = brax_rambo_config(cfg.task)
        rambo_training_params = dict(rambo_params)
        network_factory = rambo_networks.make_rambo_networks
        if "network_factory" in rambo_params:
            del rambo_training_params["network_factory"]
            network_factory = functools.partial(
                rambo_networks.make_rambo_networks,
                **rambo_params.network_factory
            )
        rambo_network = network_factory(
            observation_size=obs_size,
            action_size=act_size,
            preprocess_observations_fn=running_statistics.normalize if rambo_params.normalize_observations else None,
        )
        make_policy_fn = rambo_networks.make_inference_fn(rambo_network)
    elif cfg.policy == "flowsac":
        flowsac_params = brax_flowsac_config(cfg.task)
        flowsac_training_params = dict(flowsac_params)
        network_factory = flowsac_networks.make_flowsac_networks
        if "network_factory" in flowsac_params:
            del flowsac_training_params["network_factory"]
            network_factory = functools.partial(
                flowsac_networks.make_flowsac_networks,
                **flowsac_params.network_factory
            )
        flowsac_network = network_factory(
            observation_size=obs_size,
            action_size=act_size,
            preprocess_observations_fn=running_statistics.normalize if flowsac_params.normalize_observations else None,
            dynamics_param_size=len(env.dr_range)
        )
        make_policy_fn = flowsac_networks.make_inference_fn(flowsac_network)
    elif cfg.policy == "td3":
        if cfg.task in dm_control_suite._envs:
            td3_params = dm_control_training_config.brax_td3_config(cfg.task)
        elif cfg.task in locomotion._envs:
            td3_params= locomotion_training_config.locomotion_td3_config(cfg.task)
        td3_training_params = dict(td3_params)
        network_factory = td3_networks.make_td3_networks
        if "network_factory" in td3_params:
            del td3_training_params["network_factory"]
            network_factory = functools.partial(
                td3_networks.make_td3_networks,
                **td3_params.network_factory
            )
        td3_network = network_factory(
            observation_size=obs_size,
            action_size=act_size,
            preprocess_observations_fn=running_statistics.normalize if td3_params.normalize_observations else None, 
        )
        make_policy_fn = td3_networks.make_inference_fn(td3_network)
    # Load saved parameters
    save_dir = os.path.join(cfg.work_dir, "models")
    print(f"Loading parameters from {save_dir}")
    with open(os.path.join(save_dir, f"{cfg.policy}_params_latest.pkl"), "rb") as f:
        params = pickle.load(f)

    jit_inference_fn = jax.jit(make_policy_fn(params,deterministic=True))

    jit_reset = jax.jit(env.reset)
    jit_step = jax.jit(env.step)

    rollout = []
    rewards = []
    rng =jax.random.PRNGKey(cfg.eval_seed)
    reset_rng, rng = jax.random.split(rng)
    reset_rngs = jax.random.split(reset_rng, 10)
    for i in range(10):
        # Evaluation loop
        
        state = jit_reset(reset_rngs[i])
        total_reward = 0.0
        
        rollout = [state]

        for _ in range(env_cfg.episode_length):
            act_rng, rng = jax.random.split(rng)
            action, info = jit_inference_fn(state.obs, act_rng)
            state = jit_step(state, action)
            rollout.append(state)
            total_reward += state.reward
        rewards.append(total_reward)
    frames = env.render(rollout, camera=CAMERAS[cfg.task])
    media.show_video(frames, fps=1.0 / env.dt)
    import numpy as np
    print(f"Total reward: {np.array(rewards).mean()}")


In [5]:

cfg_path= epath.Path(".").resolve()
print(cfg_path)
cfg_path = os.path.join(cfg_path, "config.yaml")
# cfg = compose(config_name="config.yaml")
cfg = OmegaConf.load(cfg_path)
cfg.task="CheetahRun"
cfg.policy="sac"
cfg.seed=0
cfg.randomization=False

print("cfg:", cfg)
cfg = parse_cfg(cfg)
cfg.eval_seed= 55
print("work dir", cfg.work_dir)

# cfg = OmegaConf.load(cfg_path)

#843.25


/raid/users/tjrcjf410/distributionally_robust_learning/learning
cfg: {'benchmark': 'dm_control', 'task': 'CheetahRun', 'obs': 'state', 'exp_name': 'test', 'checkpoint': '???', 'eval_episodes': 1, 'eval_pi': True, 'eval_value': True, 'eval_freq': 50000, 'impl': 'warp', 'eval_with_training_env': False, 'policy': 'sac', 'asymmetric_critic': True, 'distributional_q': False, 'randomization': False, 'eval_randomization': True, 'wandb_project': 'test-algorithms', 'wandb_entity': 'tjrcjf410-seoul-national-university', 'wandb_silent': False, 'use_wandb': True, 'save_csv': True, 'comment': '', 'save_video': True, 'save_agent': True, 'seed': 0, 'work_dir': '???', 'task_title': '???', 'multitask': '???', 'tasks': '???', 'obs_shape': '???', 'action_dim': '???', 'episode_length': '???', 'obs_shapes': '???', 'action_dims': '???', 'episode_lengths': '???', 'seed_steps': '???', 'bin_size': '???', 'real_ratio': '???', 'rollout_length': '???', 'adv_weight': '???', 'batch_size': '???', 'rollout_batch_size

In [6]:
env = registry.load(cfg.task)
env._config
env.mjx_model.nbody


8

In [7]:
from learning.module.wrapper.adv_wrapper import wrap_for_adv_training


randomization_fn = registry.get_domain_randomizer_eval(cfg.task)
randomization_fn = functools.partial(randomization_fn, dr_range = env.dr_range)
rng = jax.random.PRNGKey(cfg.seed)
rng, dynamics_rng = jax.random.split(rng)
env = registry.load(cfg.task)
dr_range_low, dr_range_high = env.dr_range
env = wrap_for_adv_training(
    env,
    episode_length=1000,
    action_repeat=1,
    randomization_fn=randomization_fn,
    param_size = len(dr_range_low),
    dr_range_low=dr_range_low,
    dr_range_high=dr_range_high,
)


In [20]:
key = jax.random.PRNGKey(0)
reset_keys = jax.random.split(key, 5)
state =env.reset(reset_keys)


In [21]:

state.obs['privileged_state'][0,17:]


Array([1.5988091 , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.15238988, 0.        , 0.02539831,
       2.639688  , 1.5435146 , 1.5874476 , 1.0953975 , 1.4380753 ,
       1.2008368 , 0.8845188 ], dtype=float32)

In [22]:
state.info['dr_params']


Array([[1.5988091 , 2.639688  ],
       [1.3324037 , 3.0542428 ],
       [3.1062217 , 0.12446493],
       [0.20286039, 2.9865687 ],
       [2.5650942 , 4.7373013 ]], dtype=float32)

In [39]:
import jax.numpy as jnp
key,rng = jax.random.split(rng,2)
ns = env.step(state, jnp.zeros((5, env.action_size)), jax.random.normal(key, (5, 2)))
ns.info['dr_params']


Array([[-2.1361759 ,  0.9855011 ],
       [-0.3576046 , -1.886148  ],
       [-1.0017086 , -0.22949052],
       [-1.5266588 , -0.3220465 ],
       [ 0.6392121 , -0.34352463]], dtype=float32)

In [43]:
state.obs['privileged_state'][0,17:]


Array([-2.1361759 ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.15238988,  0.        ,  0.02539831,
        0.9855011 ,  1.5435146 ,  1.5874476 ,  1.0953975 ,  1.4380753 ,
        1.2008368 ,  0.8845188 ], dtype=float32)

In [41]:
ns.obs['privileged_state'][0,17:]


Array([-2.1361759 ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.15238988,  0.        ,  0.02539831,
        0.9855011 ,  1.5435146 ,  1.5874476 ,  1.0953975 ,  1.4380753 ,
        1.2008368 ,  0.8845188 ], dtype=float32)

In [42]:
previleged_obs_info = ns.obs['privileged_state'][:, 17:] 
state.obs['privileged_state'] = state.obs['privileged_state'].at[:, 17:].set(previleged_obs_info)
